# Project Setup

This notebook will guide you through setting up the project environment for using CrewAI. We will:

1. Install the required Python modules.
2. Set up a virtual environment.
3. Verify the installation.

In [ ]:
# Step 1: Create and activate a virtual environment
!python3 -m venv venv
!source venv/bin/activate

# Step 2: Install required Python modules
!pip install -r requirements.txt

# Step 3: Verify installation
!pip list